![The Chicago Maroon](https://chicagomaroon.com/wp-content/uploads/2022/03/Screen-Shot-2022-03-16-at-2.14.23-PM.png)

---

## Shuttle Headways & Passenger Loads Between January 1st and April 8th 2025
### *by Dariel Cruz Rodriguez*

In [1]:
from dotenv import load_dotenv
load_dotenv("./.env")
import requests
import os
import pandas as pd
import io
from datetime import datetime, timezone
import passiogo

In [2]:
API_KEY = os.getenv("API_KEY")
BASE_URL = "https://uchicagoshuttles.com"
HEADERS = {
        "key":API_KEY,
        "User-Agent": "curl",
    }

In [3]:
import pytz
from datetime import datetime

# Helper functions getApiData() and utcToCentral() adapted from https://andreithuler.com/UChicagoShuttles/api/examples/visualizations.ipynb

def getApiData(url, key):
    """
    Makes the API request and returns a Pandas dataframe
    """
    headers = {
        "key":API_KEY,
        "User-Agent": "curl",
    }
    
    response = requests.request("GET", BASE_URL+url, headers=headers)
    if response.status_code == 200:
        return(pd.read_csv(io.StringIO(response.text)))
    else:
        raise ValueError(f"ERROR making the API request: {response.text}")

def utcToCentral(datetime_utc):
    """
    Convert naive UTC datetime objects into naive US/Central datetime objects
    """
    datetime_utc = pytz.utc.localize(datetime_utc)
    datetime_central = datetime_utc.astimezone(pytz.timezone('US/Central'))
    datetime_central = datetime_central.replace(tzinfo=None)
    return datetime_central

def utcCurrent():
    """
    Return the current time in UTC.
    """
    return datetime.now(timezone.utc).strftime("%Y-%m-%d %H:%M:%S")

def centralToUtc(datetime_central_str):
    """
    Convert a Chicago (US/Central, naive) datetime string in the format "YYYY-MM-DD HH:MM"
    to its naive UTC datetime string.
    """
    
    central = pytz.timezone('US/Central')
    dt_naive = datetime.strptime(datetime_central_str, "%Y-%m-%d %H:%M:%S")
    dt_central = central.localize(dt_naive)
    dt_utc = dt_central.astimezone(pytz.utc)
    return dt_utc.strftime("%Y-%m-%d %H:%M:%S")

## Retrieving stop data

In [4]:
url = f"/api/getStops?start={centralToUtc('2025-01-01 00:00:00')}&end={utcCurrent()}"
df_allstops = getApiData(url, API_KEY)

In [5]:
df_allstops['arrivalTime'] = pd.to_datetime(df_allstops['arrivalTime'])
df_allstops['year'] = df_allstops['arrivalTime'].dt.year
df_allstops['month'] = df_allstops['arrivalTime'].dt.month
df_allstops['date'] = df_allstops['arrivalTime'].dt.day
df_allstops['hour'] = df_allstops['arrivalTime'].dt.hour
df_allstops['dayOfWeek'] = df_allstops['arrivalTime'].dt.dayofweek + 1 # (Monday -> 1)
df_allstops['periodOfDay'] = df_allstops['hour'].apply(lambda x: 1 if 4 <= x < 11 else (2 if 11 <= x < 16 else 3))
df_allstops


,id,routeName,stopName,passengerLoad,stopDurationSeconds,arrivalTime,departureTime,stopId,routeId,busId,nextStopId,year,month,date,hour,dayOfWeek,periodOfDay
0,619471,Apostolic/Drexel,Apostolic Lot,0,8,2025-01-01 05:59:58,2025-01-01 06:00:06,8614,38730,4315,133007,2025,1,1,5,3,1
1,619472,North,Reynolds Club,0,451,2025-01-01 05:52:45,2025-01-01 06:00:17,8579,38734,4314,8633,2025,1,1,5,3,1
2,619473,Apostolic/Drexel,Kenwood & 63rd,0,7,2025-01-01 06:00:18,2025-01-01 06:00:26,133007,38730,4315,8612,2025,1,1,6,3,1
3,619474,Central,Reynolds Club,0,454,2025-01-01 05:52:56,2025-01-01 06:00:31,8579,38737,4317,141882,2025,1,1,5,3,1
4,619475,North,Regenstein Library (N),0,19,2025-01-01 06:00:25,2025-01-01 06:00:44,8633,38734,4314,8634,2025,1,1,6,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
357079,976550,Apostolic,Bernard Mitchell Hospital,0,103,2025-04-09 14:49:52,2025-04-09 14:51:35,8615,38729,4329,8616,2025,4,9,14,3,2
357080,976551,Drexel,Drexel Garage,0,62,2025-04-09 14:50:36,2025-04-09 14:51:38,8611,38728,4321,8612,2025,4,9,14,3,2
357081,976552,Friend Center/Metra,59th St & Metra,0,58,2025-04-09 14:50:44,2025-04-09 14:51:42,8658,38601,4317,8612,2025,4,9,14,3,2
357082,976553,Apostolic,58th Street & Drexel,0,17,2025-04-09 14:51:35,2025-04-09 14:51:53,8616,38729,4329,8617,2025,4,9,14,3,2


## Stop locations

In [6]:
# Partially adapted from PassioGO documentation

system = passiogo.getSystemFromID(1068)
stops = system.getStops()
data = []
for stop in stops:
    data.append({"stopId": stop.id, "lat": stop.latitude, "long": stop.longitude})
df_stop_locations = pd.DataFrame(data)
df_stop_locations

,stopId,lat,long
0,10103,41.796684,-87.595753
1,8612,41.787743,-87.603607
2,8613,41.788802,-87.604871
3,8615,41.789429,-87.604871
4,8616,41.789507,-87.603924
...,...,...,...
670,cta18620,41.892662,-87.611104
671,cta18653,41.758538,-87.605213
672,cta18662,41.878133,-87.642411
673,cta18663,41.880542,-87.641183


## Ridership by stop

In [7]:
# Partially adapted from https://andreithuler.com/UChicagoShuttles/api/examples/visualizations.ipynb

df_total_ridership_per_stop = df_allstops.groupby(["stopId", "stopName", "dayOfWeek", "periodOfDay"])[["passengerLoad"]].median().reset_index()
df_total_ridership_per_stop["passengerLoad"] = df_total_ridership_per_stop["passengerLoad"].astype(int)
df_total_ridership_per_stop.drop_duplicates(inplace = True)

df_total_ridership_per_stop

,stopId,stopName,dayOfWeek,periodOfDay,passengerLoad
0,8578,Regents Park Apartments,1,1,0
1,8578,Regents Park Apartments,1,2,0
2,8578,Regents Park Apartments,1,3,0
3,8578,Regents Park Apartments,2,1,0
4,8578,Regents Park Apartments,2,3,0
...,...,...,...,...,...
1369,161849,Woodlawn Ave & 61st St,5,3,0
1370,161849,Woodlawn Ave & 61st St,6,1,0
1371,161849,Woodlawn Ave & 61st St,6,3,0
1372,161849,Woodlawn Ave & 61st St,7,1,0


In [8]:
df_total_ridership_per_stop['stopId'] = df_total_ridership_per_stop['stopId'].astype(str)
df_total_ridership_per_stop = df_total_ridership_per_stop.merge(df_stop_locations, on='stopId', how='inner')
df_total_ridership_per_stop

,stopId,stopName,dayOfWeek,periodOfDay,passengerLoad,lat,long
0,8578,Regents Park Apartments,1,1,0,41.803203,-87.585499
1,8578,Regents Park Apartments,1,2,0,41.803203,-87.585499
2,8578,Regents Park Apartments,1,3,0,41.803203,-87.585499
3,8578,Regents Park Apartments,2,1,0,41.803203,-87.585499
4,8578,Regents Park Apartments,2,3,0,41.803203,-87.585499
...,...,...,...,...,...,...,...
1369,161849,Woodlawn Ave & 61st St,5,3,0,41.784294,-87.596384
1370,161849,Woodlawn Ave & 61st St,6,1,0,41.784294,-87.596384
1371,161849,Woodlawn Ave & 61st St,6,3,0,41.784294,-87.596384
1372,161849,Woodlawn Ave & 61st St,7,1,0,41.784294,-87.596384


In [11]:
import folium
from IPython.display import display
import ipywidgets as widgets

def update_map(day, period):
    # Filter original stops data using the selected day and period
    filtered = df_allstops[
        (df_allstops['dayOfWeek'] == day) &
        (df_allstops['periodOfDay'] == period)
    ]
    
    # Group by stopId and stopName, summing the passenger load to get total ridership per stop
    df_group = filtered.groupby(['stopId', 'stopName'], as_index=False)['passengerLoad'].sum()
    
    # Convert stopId to string before merging
    df_group['stopId'] = df_group['stopId'].astype(str)
    
    # Merge with the stop locations dataframe to get the lat/long for each stop
    dff = df_group.merge(df_stop_locations, on='stopId', how='left')
    
    # Create the folium map centered on Chicago
    m = folium.Map(location=[41.792909891608296, -87.6014242653798], zoom_start=14, tiles='Cartodb Positron')
    
    # Add circle markers with a maximum radius of 3
    for _, row in dff.iterrows():
        radius = min((row['passengerLoad']**0.5) * 2 if row['passengerLoad'] != 0 else 2, 25)
        folium.CircleMarker(
            location=[row['lat'], row['long']],
            radius=radius,
            popup=f"{row['stopName']}: {row['passengerLoad']} riders",
            color='maroon',
            fill=False,
            fill_color='maroon', 
            stroke=False, 
            fill_opacity=0.7,
            stroke_opacity=0.7
        ).add_to(m)
        
    display(m)

day_dropdown = widgets.Dropdown(options=[1, 2, 3, 4, 5, 6, 7], value=1, description='Day:')
period_slider = widgets.IntSlider(value=1, min=1, max=3, step=1, description='Period:')
widgets.interact(update_map, day=day_dropdown, period=period_slider)

interactive(children=(Dropdown(description='Day:', options=(1, 2, 3, 4, 5, 6, 7), value=1), IntSlider(value=1,…

<function __main__.update_map(day, period)>